# Packages

**Modules** make reusing small chunks of code easy, but if the project grows, having many unconnected modules isn't much better. The answer to this problem is to combine related **modules** into a **package**.

A **package** is an artifact that combines one or more Python modules or packages into a single distributable file which can be uploaded to a package repository.

## What is a package?

A **module** is a file containing code. A module defines a group of usually related Python functions or objects. The name of the module is derived from the name of the file containing those functions or objects.

A **package** is a directory containing code and possibly further subdirectories. A package contains a group of usually related modules (=code files). The name of the package is derived from the name of the main package directory.

In the same way that modules pack related functions/objects together in a single file, packages group related modules together under a single directory.

## Package by example: the mathproj package

To see how packages work in practice, let's consider a design layout for a type of project addressing generalized Math concepts and constructs. Using a hierarchichal structure will be vital to keeping such a project ordered.

We will call the project as a whole, `mathproj`. A sensible way to organize its modules would be to split the project into its UI elements and its computational elements. Within the computational elements we might want to further subdivide between symbolic computation, and numeric computation.

It will be only natural to define constant files in both the symbolic and the numeric computation. Because packages are structured in subdirectories, we can define such files with the same name: `constants.py`.

```python
# numeric constants
pi = 3.141592
```

```python
class PiClass:
    def __str__(self) -> str:
        return "PI"

pi = PiClass()
```

This means that a name like `pi` can be defined and imported from two different files named `constants.py` as seen below:

![hierarchy](pics/mathproj_hierarchy.png)


There's a natural mapping from the design structure to a directory structure, which means that Python code (both inside and outside the `mathproj` package) will be able to access the two variants of `pi` as:
+ `mathproj.comp.symbolic.constants.pi`
+ `mathproj.numeric.constants.pi`

However, there are practical aspects to consider to effectively allow that.

An example mathproj package will look like the following:

![mathproj package with files](pics/mathpro_package_with_files.png)

The `__init__.py` file for the main package may look something like this:

```python
print("Hello from mathproj init")
__all__ = ["comp"]
version = 1.03
```

In turn, the `__init__.py` file for the `comp` subpackage may look something like:

```python
__all__ = ["c1"]
print("Hello from mathproj.comp init)
```

The file `c1.py` may be something like:

```python
x = 1.00
```

The `__init__.py` file for the `numeric` subpackage may simply announce itself:

```python
print("Hello from numeric init")
```

The file `n1.py` can be something like:

```python
from mathproj import version
from mathproj.comp import c1
from mathproj.comp.numeric.n2 import h

def g():
    print(f"version is {version}")
    print(h())
```

Note that it imports elements from the main package and other subpackages and defines a function.

Finally, the `n2.py` file defines the `h()` function that is imported from `n1.py`:

```python
def h():
    return "called function h() defined in module n2"
```

| EXAMPLE: |
| :------- |
| See [00_hello-mathproj](00_hello-mathproj/) for a runnable example illustrating the hierarchy structure and files mentioned above. |

### `__init__.py` files in packages

An `__init__.py` file serves two purposes:
+ Python recognizes a directory containing an `__init__.py` file as a package.
+ The code in `__init.py` is automatically executed the first time a package or subpackage is loaded, so you can use it to include your initialization logic.

For many scenarios, it is sufficient to create an empty `__init__.py`, so that the the directory is recognized as a package.

### Basic use of the package

In order to use the package from an application you just need to type:

```python
import mathproj
```

Note that importing `mathproj` will trigger the execution of `mathproj/__init__.py` but not the execution of the underlying `__init__.py` files in `comp` and `numeric` subpackages.

Because `mathproj/__init__.py` defines the variable `x` you will be able to use it by referring to `mathproj.version`.

### Loading subpackages

As explained above, loading the top-level module of a package isn't enough to load all the submodules.

As a result, if you need to invoke `g()` defined in the `n1.py` module you should do:

```python
import mathproj.comp.numeric.n1

mathproj.comp.numeric.n1.g() # this works!
```

We will see that as a side-effect of importing `n1` module, all the corresponding subpackages in the path will also be loaded and their corresponding `__init__.py` initialized.

Note however, that you need to fully qualify the function we're executing.

### `import` statements within packages

Files within a package don't automatically have access to objects defined in other files in the same package. In the same way you do in the main program, you must use `import` statements to explicitly access objects from other package files.

For example, the imports in the `n1.py` module that allows the module to invoke `h()` and print the version are:

```python
from mathproj import version
from mathproj.comp import c1
from mathproj.comp.numeric.n2 import h

def g():
    print("version is {version}")
    print(h())
```

Note how you import `version` as you would in an import statement from outside the `mathproj` package.

Because the module `n2.py` is in the same directory as `n1.py` you could have also imported `h` typing:

```python
from .n2 import h   # relative import
```

This also works to move more levels up the directory hierarchy:

```python
from ... import version
from .. import c1
from .n2 import h
```

| NOTE: |
| :---- |
| Relative imports may seem handy, but they're relative to the module's `__name__` property. Therefore, any module being executed as the main module can't use relative imports. In practice, they're not that common. |


## The `__all__` attribute

Some of the `__init__.py` files defined in `mathproj` define an attribute `__all__`. This attribute has to do with execution of statements of the form `from <package> import *`.

If present in an `__init__.py` file, `__all__` should give a list of strings defining the names that are to be imported when a `from <package> import *` is executed on that particular package. If `__all__` is not present, `from <package> import *` does nothing.

The `__all__` attribute is a great tool for package developers to clarify what's the public interface of the package.

For example, you might have something like:

```python
# vec2d.graph package
from vec2d.vector2d_graphics import (
    Arrow,
    Colors,
    draw,
)

__all__ = [
    "Arrow",
    "Colors",
    "draw",
]
```

This allow the package user to work with what is exported in the package using the following imports:

```python
from vec2d import Arrow, Colors, draw

draw(
    Arrow((2, 3), color=Colors.ORANGE),
)
```

which provides a better developer experience to the package users, as they don't need to go beyond the top-level package `vec2d` to understand how to work with the package exported symbols. It also allows the package building to choose any sort of internal organization (no matter how complex) without affecting the package users.
